In [2]:
import tensorflow as tf
import keras
from keras import layers

## Exemple 1

In [3]:
num_tags = 12  # Number of unique issue tags
num_words = 10000  # Size of vocabulary obtained when preprocessing text data
num_departments = 4  # Number of departments for predictions

title_input = keras.Input(
    shape=(None,), name="title"
)  # Variable-length sequence of ints
body_input = keras.Input(shape=(None,), name="body")  # Variable-length sequence of ints
tags_input = keras.Input(
    shape=(num_tags,), name="tags"
)  # Binary vectors of size `num_tags`

# Embed each word in the title into a 64-dimensional vector
title_features = layers.Embedding(num_words, 64)(title_input)
# Embed each word in the text into a 64-dimensional vector
body_features = layers.Embedding(num_words, 64)(body_input)

# Reduce sequence of embedded words in the title into a single 128-dimensional vector
title_features = layers.LSTM(128)(title_features)
# Reduce sequence of embedded words in the body into a single 32-dimensional vector
body_features = layers.LSTM(32)(body_features)

# Merge all available features into a single large vector via concatenation
x = layers.concatenate([title_features, body_features, tags_input])

# Stick a logistic regression for priority prediction on top of the features
priority_pred = layers.Dense(1, name="priority")(x)
# Stick a department classifier on top of the features
department_pred = layers.Dense(num_departments, name="department")(x)

# Instantiate an end-to-end model predicting both priority and department
model = keras.Model(
    inputs=[title_input, body_input, tags_input],
    outputs=[priority_pred, department_pred],
)

In [8]:
import tf2onnx
import onnx

In [11]:
body_input

<KerasTensor: shape=(None, None) dtype=float32 (created by layer 'body')>

In [12]:

input_signature = [tf.TensorSpec([None, None], tf.float32, name='title'),tf.TensorSpec([None, None], tf.float32, name='body'),tf.TensorSpec([None, None], tf.float32, name='tags')]

In [13]:
#input_signature = [tf.TensorSpec([None, 3], tf.float32, name='x')]

onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)
onnx.save(onnx_model, "model.onnx")

# Exemple 2

In [3]:
import numpy as np
inputs = {}

inputs['entree1'] = tf.keras.Input(shape=(1,), name='entree1', dtype=np.float32)
inputs['entree2'] = tf.keras.Input(shape=(1,), name='entree2', dtype=np.float32)


In [19]:
inputs_str = tf.keras.Input(shape=(1,), name="entree", dtype=tf.string)
output_str = inputs_str

In [26]:
tf.keras.Input(['test','trust'], name="entree", dtype=tf.string)

TypeError: Dimension value must be integer or None or have an __index__ method, got value ''test'' with type '<class 'str'>'

In [20]:
model = keras.Model(
    inputs=inputs_str,
    outputs=output_str
)

In [25]:
dir(inputs_str)

['__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmatmul__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '__xor__',
 '_inferred_value',
 '_keras_history',
 '_keras_mask',
 '_name',
 '_overload_all_operators',
 '_overload_operator',
 '_to_placeholder',
 '_type_spec',
 'dtype',
 'experimental_ref',
 'from_te

In [21]:
import pandas as pd

df = pd.DataFrame(data={'entree1':['titi','toto'],'entree2':[2.3,6.3]})
features_dict = {name: np.array(value) 
                         for name, value in df.items()}

In [23]:
model.predict(features_dict['entree1'])

1/1 [==============================] - 0s 35ms/step


array([[b'titi'],
       [b'toto']], dtype=object)

In [45]:
tf_constant = tf.convert_to_tensor([['tot','tutu']])

In [48]:
tf.keras.layers.Layer(
    
)

In [49]:
y = tf.constant([[2.0], [3.0]])

In [51]:
def simple_relu(x):
  if tf.greater(x, 0):
    return 'toto'
  else:
    return 'titi'

# `tf_simple_relu` is a TensorFlow `Function` that wraps `simple_relu`.
tf_simple_relu = tf.function(simple_relu)


In [52]:
tf_simple_relu(y)

ValueError: in user code:

    File "<ipython-input-51-615551f3c4b4>", line 2, in simple_relu  *
        if tf.greater(x, 0):

    ValueError: condition of if statement expected to be `tf.bool` scalar, got Tensor("Greater:0", shape=(2, 1), dtype=bool); to check for None, use `is not None`


In [47]:
model = keras.Model(
    inputs=inputs['entree1'],
    outputs=tf_constant,
)

ValueError: Output tensors of a Functional model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: [[b'tot' b'tutu']]

In [30]:
array = np.array([[1,2,3],[3,4,5],[5,6,7]])
tensor = tf.convert_to_tensor(array)
print("Converted numpy array into tensor:")
print(tensor)

Converted numpy array into tensor:
tf.Tensor(
[[1 2 3]
 [3 4 5]
 [5 6 7]], shape=(3, 3), dtype=int32)


In [27]:
features_dict

{'entree1': array(['titi', 'toto'], dtype=object),
 'entree2': array([2.3, 6.3])}

In [31]:
inputs['entree1']

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'entree1')>

In [7]:
from keras import backend as K

def customCall(inputs):

    ones = K.ones_like(inputs)
    lower = K.exp(inputs)
    higher = K.exp(1-inputs)

    outputs = K.switch(K.greater(inputs,1), higher, ones)
    outputs = K.switch(K.less_equal(inputs,0), lower, outputs)

    return outputs

In [28]:
input

<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x0000018E37749730>>

In [32]:
ones = K.ones_like(inputs['entree1'])

In [33]:
ones

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'tf.ones_like')>

In [34]:
con_lis = tf.convert_to_tensor(inputs['entree1'])

In [38]:
list_p = [['tut','tat']]

In [39]:
tf.convert_to_tensor(list_p)

<tf.Tensor: shape=(1, 2), dtype=string, numpy=array([[b'tut', b'tat']], dtype=object)>

In [35]:
con_lis

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'tf.convert_to_tensor')>

In [10]:
resu = layers.Lambda(customCall)(inputs['entree1'])
model = keras.Model(
    inputs=inputs['entree1'],
    outputs=resu,
)



In [11]:
model = keras.Model(
    inputs=inputs['entree1'],
    outputs=resu,
)

In [16]:
import pandas as pd

df = pd.DataFrame(data={'entree1':[0.3,7.3],'entree2':[2.3,6.3]})
features_dict = {name: np.array(value) 
                         for name, value in df.items()}

In [17]:
model.predict(features_dict['entree1'])

1/1 [==============================] - 0s 18ms/step


array([[1.       ],
       [0.0018363]], dtype=float32)

In [4]:
a = tf.Variable(initial_value=[0, 0, 4, 6, 1, 2, 4, 0])
mask = tf.greater_equal(a, 2)  # [False False  True  True False  True  True False]
indexes = tf.where(mask)  # [[2] [3] [5] [6]], shape=(4, 1)
b = tf.scatter_update(a, mask, tf.constant(1500))

AttributeError: module 'tensorflow' has no attribute 'scatter_update'

In [18]:
tf.add(inputs['entree1'], inputs['entree2'])

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'tf.math.add_2')>

In [65]:
from keras import backend as K

def rescale(inputs):
    mask = K.cast(inputs['entree1'] >= inputs['entree2'], dtype=K.floatx())
    print(mask)
    return mask 


In [57]:
scaled = layers.Lambda(rescale)(inputs)

In [78]:
inputs['entree1'].shape

TensorShape([None, 1])

In [82]:
f1 = lambda: tf.constant(2,shape=[2, 1])
f2 = lambda: tf.constant(23,shape=[2, 1])
r = tf.case([(tf.less(inputs['entree1'], inputs['entree2']), f1)], default=f2)

In [87]:
tf.less(inputs['entree1'], inputs['entree2'])

<KerasTensor: shape=(None, 1) dtype=bool (created by layer 'tf.math.less_13')>

In [55]:
import tensorflow.keras.backend as K
@tf.function
def custom_layer(inputs):

    array = inputs['entree1'].numpy()
    print(array)

    v  =  inputs['entree1']>inputs['entree2']
     
    return v

In [49]:
lambda_layer = layers.Lambda(custom_layer, name="lambda_layer")(inputs)


AttributeError: Exception encountered when calling layer "lambda_layer" (type Lambda).

in user code:

    File "<ipython-input-48-01738d157d21>", line 4, in custom_layer  *
        array = inputs['entree1'].numpy()

    AttributeError: 'Tensor' object has no attribute 'numpy'


Call arguments received by layer "lambda_layer" (type Lambda):
  • inputs={'entree1': 'tf.Tensor(shape=(None, 1), dtype=float32)', 'entree2': 'tf.Tensor(shape=(None, 1), dtype=float32)'}
  • mask=None
  • training=None

In [44]:
lambda_layer

<KerasTensor: shape=() dtype=int32 inferred_value=[None] (created by layer 'lambda_layer')>

In [83]:
model = keras.Model(
    inputs=inputs,
    outputs=r,
)

In [84]:
import pandas as pd

df = pd.DataFrame(data={'entree1':[1.3,7.3],'entree2':[2.3,6.3]})
features_dict = {name: np.array(value) 
                         for name, value in df.items()}

In [85]:
model.predict(features_dict)

InvalidArgumentError: Graph execution error:

The second input must be a scalar, but it has shape [2]
	 [[{{node model_11/tf.case_5/case/cond/switch_pred/_2}}]] [Op:__inference_predict_function_3872]

In [21]:
result = tf.cond(tf.less(inputs['entree1'] < inputs['entree2'], lambda: tf.add(inputs['entree1'], inputs['entree2']), lambda: tf.square(inputs['entree2'])))

TypeError: Exception encountered when calling layer "tf.math.less_4" (type TFOpLambda).

Value passed to parameter 'x' has DataType bool not in list of allowed values: float32, float64, int32, uint8, int16, int8, int64, bfloat16, uint16, float16, uint32, uint64

Call arguments received by layer "tf.math.less_4" (type TFOpLambda):
  • x=tf.Tensor(shape=(None, 1), dtype=bool)
  • y=<function <lambda> at 0x000001D9B2E58CA0>
  • name=<function <lambda> at 0x000001D9B6BA7E50>